In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from google.colab import files
from collections import defaultdict
import smtplib
from email.message import EmailMessage
import ssl
import getpass


# Load reconciliation data
def load_data(file_path):
    df = pd.read_csv(file_path)
    df = df.dropna(subset=["As of Date"])  # Remove empty rows
    df["As of Date"] = pd.to_datetime(df["As of Date"], format="%m/%d/%Y")
    df[["Company", "Account", "AU", "IHub Balance"]] = df[["Company", "Account", "AU", "IHub Balance"]].astype("Int64")
    if df["Currency"].nunique() == 1:
       df = df.drop(columns=["Currency"])
    return df

# Detect anomalies using Isolation Forest
def detect_anomalies(df):
    numerical_features = ["GL Balance", "IHub Balance", "Balance Difference"]
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(df[numerical_features])
    iso_forest = IsolationForest(contamination=0.1, random_state=42)
    df["Anomaly"] = iso_forest.fit_predict(scaled_data) == -1
    return df

# Function to classify anomalies
def classify_anomaly(row):
    if abs(row["Balance Difference"]) > 50000:
        return "Large Balance Difference"
    elif row["IHub Balance"] == 0 and abs(row["GL Balance"]) > 10000:
        return "Zero IHub Balance"
    elif (row["GL Balance"] > 0 and row["IHub Balance"] < 0) or (row["GL Balance"] < 0 and row["IHub Balance"] > 0):
        return "Unexpected Credit/Debit Mismatch"
    elif 1 < abs(row["Balance Difference"]) < 100:
        return "Tolerance Breach"
    else:
        return "New Reason Detected"

# Apply clustering and anomaly detection
def detect_patterns_and_anomalies(df):
    numerical_features = ["GL Balance", "IHub Balance", "Balance Difference"]
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(df[numerical_features])

    # Clustering with K-Means
    kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
    df["Cluster"] = kmeans.fit_predict(scaled_data)

    # Anomaly detection using Isolation Forest
    iso_forest = IsolationForest(contamination=0.1, random_state=42)
    df["Anomaly"] = iso_forest.fit_predict(scaled_data)
    df["Anomaly"] = df["Anomaly"] == -1

    # Classify anomalies
    df["Anomaly Reason"] = df.apply(classify_anomaly, axis=1)
    return df

# Generate AI-powered resolution insights with human-in-the-loop learning
def generate_resolution_summaries(df, history, feedback_log):
    resolution_summaries = []
    for _, row in df[df["Anomaly"]].iterrows():
        key = (row["Account"], row["Balance Difference"])
        past_resolutions = history.get(key, [])

        if key in feedback_log:
            resolution = f"Suggested by operator: {feedback_log[key]}"
        elif past_resolutions:
            resolution = f"Similar breaks resolved as: {past_resolutions[-1]}"
        else:
            resolution = "No historical resolution found. Manual review required."

        resolution_summaries.append({
            "As of Date": row["As of Date"],
            "Account": row["Account"],
            "Balance Difference": row["Balance Difference"],
            "Suggested Resolution": resolution
        })

    send_email_notification(resolution_summaries)
    return pd.DataFrame(resolution_summaries)


# Send email notifications via Gmail
def send_email_notification(resolution_summaries):
    sender_email = input("Enter Sender Email Address:")
    sender_password = getpass.getpass("Enter Sender Email Password (App Password if you are using Gmail): ")
    recipient_email = input("Enter Recipient Email Address:")

    msg = EmailMessage()
    msg["Subject"] = "Consolidated Reconciliation Anomalies Report"
    msg["From"] = sender_email
    msg["To"] = recipient_email
    email_content = "Reconciliation Breaks Detected:\n\n"
    for summary in resolution_summaries:
        email_content += (f"Date: {summary['As of Date']}, Account: {summary['Account']}, "
                          f"Balance Difference: {summary['Balance Difference']}, "
                          f"Resolution: {summary['Suggested Resolution']}\n")

    msg.set_content(email_content)

    context = ssl.create_default_context()
    with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
        server.login(sender_email, sender_password)
        server.send_message(msg)
    print(f"Consolidated email notification sent")

# Simulated historical resolutions for learning
historical_resolutions = defaultdict(list)
historical_resolutions[(1619205, -40000)].append("Reclassified expense category")
historical_resolutions[(1619206, 80000)].append("Journal entry adjustment required")

# Feedback log for human-in-the-loop learning
feedback_log = {}

def update_feedback(account, balance_diff, feedback):
    feedback_log[(account, balance_diff)] = feedback
    print(f"Feedback recorded for Account {account}, Balance Diff {balance_diff}: {feedback}")

# Run the process

# Upload file in Google Colab
uploaded = files.upload()
file_path = list(uploaded.keys())[0]

df = load_data(file_path)
df = detect_patterns_and_anomalies(df)
df = detect_anomalies(df)
resolution_df = generate_resolution_summaries(df, historical_resolutions, feedback_log)

# Display AI-generated resolutions
print(resolution_df)

# Display anomalies
detected_anomalies = df[df["Anomaly"]]
print(detected_anomalies[["As of Date", "Account", "GL Balance", "IHub Balance", "Balance Difference", "Cluster", "Anomaly Reason"]])


Saving Sample Reconciliation Data.csv to Sample Reconciliation Data (2).csv
Enter Sender Email Address:shanthi.varadharaj1@gmail.com
Enter Sender Email Password (App Password if you are using Gmail): ··········
Enter Recipient Email Address:viji524@gmail.com
Consolidated email notification sent
  As of Date  Account  Balance Difference  \
0 2024-04-30  1619205            -40000.0   
1 2024-04-30  1619288             80000.0   
2 2024-03-31  1619288             70000.0   

                                Suggested Resolution  
0  Similar breaks resolved as: Reclassified expen...  
1  No historical resolution found. Manual review ...  
2  No historical resolution found. Manual review ...  
  As of Date  Account  GL Balance  IHub Balance  Balance Difference  Cluster  \
0 2024-04-30  1619205     20000.0         60000            -40000.0        2   
6 2024-04-30  1619288     80000.0             0             80000.0        0   
7 2024-03-31  1619288     80000.0         10000             700